In [24]:
#группа 1 -контроль группа (все по-старому) 2 - новые алгоритмы(таргет)
#группа 0 -контроль группа (все по-старому) 3 - новые алгоритмы(таргет
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandahouse
import numpy as np
import hashlib #делать хэши
#import swifter #ускорение метода apply
from scipy import stats #статистика
%matplotlib inline

In [25]:
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20220520'
     }

q = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM simulator_20220520.feed_actions 
WHERE toDate(time) between '2022-04-24' and '2022-04-30'
    and exp_group in (0,1,2,3)
GROUP BY exp_group, user_id
"""
df = pandahouse.read_clickhouse(q, connection=connection)

In [26]:
#Считаем общий CTR в контрольных группах 0 и 1
#𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙=𝑠𝑢𝑚(𝑙𝑖𝑘𝑒𝑠)/𝑠𝑢𝑚(𝑣𝑖𝑒𝑤𝑠)
CTR_control_0= df[df.exp_group == 0].likes.sum()/df[df.exp_group == 0].views.sum()
CTR_control_1= df[df.exp_group == 1].likes.sum()/df[df.exp_group == 1].views.sum()

In [27]:
#Посчитаем в обеих группах поюзерную метрику  𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠=𝑙𝑖𝑘𝑒𝑠−𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙∗𝑣𝑖𝑒𝑤𝑠
def get_linearized (user_likes,user_views, CTR_control):
    linearized_likes = (user_likes-CTR_control*user_views) 
    return linearized_likes

In [28]:
group0=df[df.exp_group == 0].copy()
group1=df[df.exp_group == 1].copy()
group2=df[df.exp_group == 2].copy()
group3=df[df.exp_group == 3].copy()

In [29]:
# Группы 0 и 3, где 0 -контроль группа
group0['linearized_likes'] = df.apply(
    lambda x: get_linearized(x['likes'],x['views'], CTR_control_0), axis=1) 

group3['linearized_likes'] = df.apply(
    lambda x: get_linearized(x['likes'],x['views'], CTR_control_0), axis=1) 

In [30]:
#Группы 1 и 2, где  1 -контроль группа
group1['linearized_likes'] = df.apply(
    lambda x: get_linearized(x['likes'],x['views'], CTR_control_1), axis=1) 

group2['linearized_likes'] = df.apply(
    lambda x: get_linearized(x['likes'],x['views'], CTR_control_1), axis=1) 

In [31]:
#t-test до линеаризации, группа 0 и 3 
print(stats.ttest_ind(
        df[df.exp_group == 0].ctr,
        df[df.exp_group == 3].ctr,
          equal_var=False) )

Ttest_indResult(statistic=-13.896870721904069, pvalue=1.055849414662529e-43)


In [32]:
#t-test после линеаризации, группа 0 и 3 
print(stats.ttest_ind(
                group0.linearized_likes,
                group3.linearized_likes,
                  equal_var=False) )

Ttest_indResult(statistic=-15.214995460903827, pvalue=5.4914249479690016e-52)


In [33]:
#t-test до линеаризации, группа 1 и 2 
print(stats.ttest_ind(
        df[df.exp_group == 1].ctr,
        df[df.exp_group == 2].ctr,
          equal_var=False) )

Ttest_indResult(statistic=0.7094392041270485, pvalue=0.4780623130874935)


In [34]:
#t-test после линеаризации, группа 1 и 2
print(stats.ttest_ind(
                group1.linearized_likes,
                group2.linearized_likes,
                  equal_var=False) )

Ttest_indResult(statistic=6.122579994775974, pvalue=9.439432187037644e-10)


Вывод:
В группе 0 и 3 p-value стало ниже  после линеаризации.

В группе 1 и 2 до линеаризации не было обнаружено статистических различий, после линеаризации p-value cтало значительно ниже 0.05. Т.е были обнаружены статистические различия.